In [1]:
import os

print("Before deletion:", os.environ.get("OLLAMA_HOST"))

if "OLLAMA_HOST" in os.environ:
    del os.environ["OLLAMA_HOST"]
    
print("After deletion:", os.environ.get("OLLAMA_HOST"))  # None이면 삭제됨
print("OLLAMA_HOST" in os.environ)  # False이면 삭제된 상태

Before deletion: 0.0.0.0
After deletion: None
False


In [2]:
from tqdm.auto import tqdm
import traceback

In [3]:
import ollama
from ollama import chat
from ollama import ChatResponse

In [4]:
from arc_utils import *

In [5]:
challenge_data, solution_data = load_data()

In [6]:
ollama.list()

ListResponse(models=[Model(model='gemma3:27b-it-qat', modified_at=datetime.datetime(2025, 6, 19, 10, 9, 47, 320527, tzinfo=TzInfo(+09:00)), digest='29eb0b9aeda35295ed728124d341b27e0c6771ea5c586fcabfb157884224fa93', size=18087370898, details=ModelDetails(parent_model='', format='gguf', family='gemma3', families=['gemma3'], parameter_size='27.4B', quantization_level='Q4_0')), Model(model='qwen2.5:latest', modified_at=datetime.datetime(2025, 5, 14, 20, 34, 30, 946725, tzinfo=TzInfo(+09:00)), digest='845dbda0ea48ed749caafd9e6037047aa19acfcfd82e704d7ca97d631a0b697e', size=4683087332, details=ModelDetails(parent_model='', format='gguf', family='qwen2', families=['qwen2'], parameter_size='7.6B', quantization_level='Q4_K_M')), Model(model='gemma3:latest', modified_at=datetime.datetime(2025, 5, 8, 9, 54, 29, 315063, tzinfo=TzInfo(+09:00)), digest='a2af6cc3eb7fa8be8504abaf9b04e88f17a119ec3f04a3addf55f92841195f5a', size=3338801804, details=ModelDetails(parent_model='', format='gguf', family='gemm

In [7]:
# challenge_data, solution_data = load_data("evaluation")

In [8]:
len(challenge_data)

1000

In [9]:
for key, data in challenge_data.items():
    if (len(data['test']) == 3):
        print(data['test'])
        break
    # print(key)
    # print(data)
    # print(len(data['test']))
    # break

[{'input': [[7, 7, 0], [7, 0, 7], [0, 7, 0]]}, {'input': [[0, 8, 0], [8, 8, 8], [0, 8, 0]]}, {'input': [[2, 0, 2], [0, 2, 0], [2, 0, 2]]}]


In [10]:
data

{'train': [{'input': [[0, 1, 1], [0, 1, 1], [1, 0, 0]], 'output': [[3]]},
  {'input': [[0, 8, 8], [0, 8, 8], [8, 0, 0]], 'output': [[3]]},
  {'input': [[0, 5, 0], [5, 5, 5], [0, 5, 0]], 'output': [[6]]},
  {'input': [[8, 0, 8], [0, 8, 0], [8, 0, 8]], 'output': [[2]]},
  {'input': [[5, 0, 5], [0, 5, 0], [5, 0, 5]], 'output': [[2]]},
  {'input': [[4, 4, 0], [4, 0, 4], [0, 4, 0]], 'output': [[1]]},
  {'input': [[5, 5, 0], [5, 0, 5], [0, 5, 0]], 'output': [[1]]}],
 'test': [{'input': [[7, 7, 0], [7, 0, 7], [0, 7, 0]]},
  {'input': [[0, 8, 0], [8, 8, 8], [0, 8, 0]]},
  {'input': [[2, 0, 2], [0, 2, 0], [2, 0, 2]]}]}

In [11]:
results_path = 'results.jsonl'
processed_keys = set()

# 이전에 저장된 results.jsonl이 있으면 읽어서 key 목록 확보
if os.path.exists(results_path):
    with open(results_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                record = json.loads(line)
                processed_keys.add(record['key'])
            except Exception:
                continue  # 혹시 잘못된 줄이 있어도 무시

# main loop
with open(results_path, 'a', encoding='utf-8') as f_out:
    for key, data in tqdm(list(challenge_data.items())[:100], desc="Processing"):
        if key in processed_keys:
            continue  # 이미 처리한 항목이면 건너뜀

        answer = solution_data.get(key)
        prediction = ''
        error_message = None

        try:
            stream = chat(
                model='gemma3:27b-it-qat',
                messages=[
                    {
                        'role': 'system',
                        'content': (
                            '다음 입출력을 보고 test의 output으로 적합한 output을 출력하라. '
                            'json parsing이 가능한 형태로, output만 출력할 것.'
                        )
                    },
                    {'role': 'user', 'content': f'{str(data)}'}
                ],
                stream=True,
            )

            for chunk in stream:
                content = chunk['message']['content']
                print(content, end='', flush=True)
                prediction += content

        except Exception as e:
            error_message = str(e)
            print(f"\n❌ 오류 발생: {e}")
            traceback.print_exc()

        print("\n")  # 줄바꿈

        result = {
            'key': key,
            'input': data,
            'answer': answer,
            'prediction': prediction.strip(),
            'error': error_message,
        }

        # 결과 한 줄씩 바로 저장
        f_out.write(json.dumps(result, ensure_ascii=False) + '\n')
        f_out.flush()  # 크래시 시 유실 방지

        processed_keys.add(key)  # 실시간으로 키 추가

Processing:   0%|          | 0/100 [00:00<?, ?it/s]

```json
{
  "test": [
    {
      "output": [
        [3, 2, 3, 2, 3, 2],
        [7, 8, 7, 8, 7, 8],
        [2, 3, 2, 3, 2, 3],
        [8, 7, 8, 7, 8, 7],
        [3, 2, 3, 2, 3, 2],
        [7, 8, 7, 8, 7, 8]
      ]
    }
  ]
}
```

```json
{
  "test": [
    {
      "input": [
        [
          7,
          0,
          7
        ],
        [
          7,
          0,
          7
        ],
        [
          7,
          7,
          0
        ]
      ],
      "output": [
        [
          7,
          0,
          7,
          7,
          0,
          7,
          7,
          0,
          7
        ],
        [
          7,
          0,
          7,
          7,
          0,
          7,
          7,
          0,
          7
        ],
        [
          7,
          7,
          0,
          7,
          7,
          0,
          7,
          7,
          0
        ],
        [
          7,
          0,
          7,
          7,
          0,
          7,
          7,
  

In [ ]:
with open('../arc_agi_rules.txt', 'r', encoding='utf-8') as f:
    rules = ''.join(f.readlines())

In [ ]:
results_path = 'results_with_rules.jsonl'
processed_keys = set()

# 이전에 저장된 results.jsonl이 있으면 읽어서 key 목록 확보
if os.path.exists(results_path):
    with open(results_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                record = json.loads(line)
                processed_keys.add(record['key'])
            except Exception:
                continue  # 혹시 잘못된 줄이 있어도 무시

# main loop
with open(results_path, 'a', encoding='utf-8') as f_out:
    for key, data in tqdm(list(challenge_data.items())[:100], desc="Processing"):
        if key in processed_keys:
            continue  # 이미 처리한 항목이면 건너뜀

        answer = solution_data.get(key)
        prediction = ''
        error_message = None

        try:
            stream = chat(
                model='gemma3:27b-it-qat',
                messages=[
                    {
                        'role': 'system',
                        'content': (
                            '다음 입출력을 보고 test의 output으로 적합한 output을 출력하라. ' + \
                            'json parsing이 가능한 형태로, output만 출력할 것. ' + \
                            '문제를 풀기 위해 다음과 같은 규칙 중 일부를 적용할 수 있다.\n' + \
                            rules
                        )
                    },
                    {'role': 'user', 'content': f'{str(data)}'}
                ],
                stream=True,
            )

            for chunk in stream:
                content = chunk['message']['content']
                print(content, end='', flush=True)
                prediction += content

        except Exception as e:
            error_message = str(e)
            print(f"\n❌ 오류 발생: {e}")
            traceback.print_exc()

        print("\n")  # 줄바꿈

        result = {
            'key': key,
            'input': data,
            'answer': answer,
            'prediction': prediction.strip(),
            'error': error_message,
        }

        # 결과 한 줄씩 바로 저장
        f_out.write(json.dumps(result, ensure_ascii=False) + '\n')
        f_out.flush()  # 크래시 시 유실 방지

        processed_keys.add(key)  # 실시간으로 키 추가

In [ ]:
# for key, data in challenge_data.items()[:100]:

#     answer = solution_data[key]
    
#     stream = chat(
#         model='gemma3:27b-it-qat',
#         messages=[
#             {'role': 'system', 'content': ('다음 입출력을 보고 test의 output으로 적합한 output을 출력하라.'
#                                             'json parsing이 가능한 형태로, output만 출력할 것.')},
#             {'role': 'user', 'content': f'{str(data)}'}],
#         stream=True,
#     )
    
#     for chunk in stream:
#         print(chunk['message']['content'], end='', flush=True)

In [ ]:
ollama.list()

In [ ]:
# response: ChatResponse = chat(model='gemma3', messages=[
#   {
#     'role': 'user',
#     'content': '블루 아카이브가 뭐지?',
#   },
# ])
# print(response['message']['content'])
# # print(response.message.content)

In [ ]:
# response: ChatResponse = chat(model='gemma3:27b-it-qat', messages=[
#   {
#     'role': 'user',
#     'content': '블루 아카이브가 뭐지?',
#   },
# ])
# print(response['message']['content'])
# # print(response.message.content)

In [ ]:
from ollama import chat

stream = chat(
    model='gemma3:27b-it-qat',
    messages=[{'role': 'user', 'content': '블루 아카이브가 뭐지?'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

In [ ]:
# import requests

# try:
#     response = requests.get("http://localhost:11434")
#     print(response.status_code)
#     print(response.text)
# except requests.exceptions.ConnectionError:
#     print("❌ Ollama 서버에 연결할 수 없습니다. 서버가 실행 중인지 확인하세요.")

In [ ]:
# import requests

# response = requests.get("http://localhost:11434/api/tags")
# print(response.status_code)
# print(response.json())